In [ ]:
import requests
import json
import gradio as gr
import base64
from PIL import Image
from io import BytesIO

# Azure OpenAI 및 Azure Cognitive Search 설정
ENDPOINT = "your-endpoint"
API_KEY = "your-api-key"
AI_SEARCH_ENDPOINT = "your-search-endpoint"
AI_SEARCH_API_KEY = "your-search-api-key"
AI_SEARCH_INDEX = "your-search-index"
AI_SEARCH_SEMANTIC = "your-search-semantic"

def chat(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY
    }
    body = {
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": AI_SEARCH_ENDPOINT,
                    "index_name": AI_SEARCH_INDEX,
                    "semantic_configuration": AI_SEARCH_SEMANTIC,
                    "query_type": "simple",
                    "fields_mapping": {},
                    "in_scope": True,
                    "role_information": "It can create new animals with the characteristics of animals and give me information.\nIf you input a user query in Korean, it will translate it into English and input it.\nThe output data will be translated into Korean.",
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": AI_SEARCH_API_KEY
                    },
                    "key": AI_SEARCH_API_KEY,
                    "indexName": AI_SEARCH_INDEX
                }
            }
        ],
        "messages": [
            {"role": "system", "content": "It can create new animals with the characteristics of animals and give me information.\nIf you input a user query in Korean, it will translate it into English and input it.\nThe output data will be translated into Korean."},
            {"role": "user", "content": prompt},
            {
              "role": "assistant",
              "content": "###**Horned Equus Characteristics**  \n- **Height**: 140–160 cm (similar to Arabian horses)  \n- **Weight**: 380–1000 kg  \n- **Color**: Various (maintaining horse-like coloration)  \n- **Lifespan**: 25–30 years  \n- **Diet**: Herbivore  \n- **Habitat**: Grasslands, mountainous regions (considering both deer and horses)  \n- **Horn Shape**: Large, curved, similar to elk or deer  \n- **Average Speed**: 50–60 km/h (slightly lower than horses due to horn weight)  \n- **Top Speed**: 60–70 km/h  \n- **Social Structure**: Herd-based  \n\nNow, I have generated an image of the **Horned Equus!** 🦄✨  \nIt features a strong, muscular body combined with gracefully curved horns, set against a vast grassland and mountainous backdrop. Let me know if you'd like any modifications or a different artistic style! 😊\n\n\n\n\n\n\n\n"
            },
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        "stream": True
    }
    
    response = requests.post(ENDPOINT, headers=headers, json=body, stream=True)
    
    if response.status_code == 200:
        decoded_text = ""
        for line in response.iter_lines():
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    try:
                        json_obj = json.loads(line[6:])
                        for choice in json_obj.get("choices", []):
                            if "delta" in choice and "content" in choice["delta"]:
                                decoded_text += choice["delta"]["content"]
                    except json.JSONDecodeError:
                        continue
        return decoded_text
    else:
        return f"Error: {response.status_code}"


import requests  
import time
# Define the endpoint and headers  
endpoint = "your-dall-e-endpoint"  

def image(content):
      headers = {  
          "api-key": "your-dall-e-api-key"  
      }  
        
      # Define the body for the request  
      body = {  
          "prompt": content,  
          "n": 1,  
          "size": "1024x1024"  
      }  
        
      # Make the initial request to generate the image  
      response = requests.post(endpoint, headers=headers, json=body)  
        
      # Check if the request was successful  
      if response.status_code == 202:  # Accepted  
          operation_location = response.headers.get('operation-location')  
            
          if operation_location:  
              # Poll for the result  
              while True:  
                  image_response = requests.get(operation_location, headers=headers)  
                  image_response_json = image_response.json()  
                    
                  print("Status:", image_response_json['status'])  
                    
                  if image_response_json['status'] == "succeeded":  
                      image_url = image_response_json['result']['data'][0]['url']  
                      break  
                  elif image_response_json['status'] == "failed":  
                      print("Image generation failed:", image_response_json)  
                      exit()  
                    
                  time.sleep(1)  
          else:  
              print("Operation location not found in the response.")  
      else:  
          if 'data' in response.json() and response.json()['data']:  
              image_url = response.json()['data'][0]['url']
              image_response = requests.get(image_url)
              image = Image.open(BytesIO(image_response.content))
              return image
          

# 여러 이미지를 Base64로 인코딩
with open(r"blocks.jpg", "rb") as f:
    main_bg_data = f.read()
    
with open(r"row1.jpg", "rb") as f:
    row1_bg_data = f.read()
    
with open(r"row2.jpg", "rb") as f:
    row2_bg_data = f.read()

main_bg_base64 = base64.b64encode(main_bg_data).decode('utf-8')
row1_bg_base64 = base64.b64encode(row1_bg_data).decode('utf-8')
row2_bg_base64 = base64.b64encode(row2_bg_data).decode('utf-8')

# CSS
css = f"""
/* 메인 컨테이너 배경 */
.gradio-container {{
    background-image: url('data:image/jpeg;base64,{main_bg_base64}');
    background-size: cover;
    background-position: center;
}}

/* 첫 번째 Row 배경 */
.input-row {{
    background-image: url('data:image/jpeg;base64,{row1_bg_base64}');
    background-size: cover;
    background-position: center;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
}}

/* 두 번째 Row 배경 */
.output-row {{
    background-image: url('data:image/jpeg;base64,{row2_bg_base64}');
    background-size: cover;
    background-position: center;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
}}
"""


with gr.Blocks(css=css) as demo:
    gr.Markdown("## 🦁 나만의 동물 만들기!")

    with gr.Row(elem_classes=["input-row"]):
        prompt_box = gr.Textbox(label="📝 PROMPT", placeholder="새로운 동물의 특성을 입력하세요", scale=6)
        send_button = gr.Button("🚀 생성")
      
    with gr.Row(elem_classes=["output-row"]):
      img_output = gr.Image(label="🖼️ 생성된 동물 이미지")
      md_output = gr.Markdown("📜 여기에 생성된 동물이 표시됩니다.")


    def generate_animal(prompt):
        animal_info = chat(prompt)  # ✅ 동물 정보 생성
        return animal_info, animal_info  # ✅ 이미지 함수에도 전달해야 하므로 두 번 반환

    # 2️⃣ image() 실행 → 이미지 출력
    def generate_image(animal_info):
        img_url = image(animal_info)  # ✅ 동물 정보를 바탕으로 이미지 생성
        return img_url

    # ✅ 첫 번째 클릭: 동물 정보 생성
    send_button.click(generate_animal, inputs=[prompt_box], outputs=[md_output, md_output])\
               .then(generate_image, inputs=[md_output], outputs=[img_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7910

To create a public link, set `share=True` in `launch()`.
